<a href="https://colab.research.google.com/github/mgfrantz/CTME-llm-lecture-resources/blob/main/prototyping_ai/02_rag_and_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval Augmented Generation

![](https://www.dailydoseofds.com/content/images/2024/10/rag.gif)
Source: [Daily Dose of DS](https://www.dailydoseofds.com/a-crash-course-on-building-rag-systems-part-1-with-implementations/)

In [ ]:
%load_ext rich
%load_ext autoreload
%autoreload 2

# Introduction to RAG with LlamaIndex

LlamaIndex is a library for working with large language models.
One of its main strengths is its ability to ingest documents into a vector index and use them to answer questions.
This is known as Retrieval Augmented Generation (RAG).

To start, we will use a low-code, high-level abstraction to build a basic PDF question-answering system.
We will read in PDFs, split them into chunks, embed them, and store them in a vector database.
Then, we will use an abstraction known as a `QueryEngine` that implements RAG to answer questions about the documents.

In [1]:
# If we're in colab, use userdata to get the OPENAI_API_KEY
import os
from rich import print
from pathlib import Path

try:
    from google.colab import userdata
    print("Colab detected - setting up environment")
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    %pip install llama-index \
        "openai==1.55.3" \
        "httpx==0.27.2" \
        llama-index-readers-web \
        thefuzz \
        gradio \
        chromadb \
        llama-index-embeddings-huggingface \
        llama-index-vector-stores-chroma \
        llama-index-retrievers-bm25 \
        llama-index-llms-gemini \
        docling \
        llama-index-readers-docling \
        llama-index-node-parser-docling \
        llama-index-utils-workflow \
        llama-index-tools-yahoo-finance \
        llama-index-tools-code-interpreter

except:
    print("Not in colab - using local environment variables.")
    from dotenv import load_dotenv
    load_dotenv("../.env")


Colab detected - setting up environment

In [ ]:
import os
import requests

# Create data directory if it doesn't exist
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Download the PDF file
pdf_url = "https://arxiv.org/pdf/2407.21783"
pdf_path = os.path.join(data_dir, "2407.21783.pdf")

if not os.path.exists(pdf_path):
    response = requests.get(pdf_url)
    with open(pdf_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded PDF to {pdf_path}")
else:
    print(f"PDF already exists at {pdf_path}")


The first thing we need to do is to load the data.
For general documents like PDFs, LlamaIndex provides a nice abstraction known as a `SimpleDirectoryReader` that can load data from a directory.
In the cell below, we use it to load the data from the `data` directory.

In [ ]:
from llama_index.core.readers import SimpleDirectoryReader
documents = SimpleDirectoryReader(data_dir).load_data()

Let's take a look at the first document.
Most prominently, we get the text of the document.
By default, we also get a lot of useful information, like the page number file name, file path, type, size, etc.
When we load lots of documents, this type of information becomes important to keep track of.

In [ ]:
print(documents[0])

Now that we've loaded the data, we need to vectorize it.
We will use a combination of an embedding model and a vector database to store the vectors.
In the cell below, we use a HuggingFace embedding model to embed the documents.
We also use torch to determine the device to use for the embedding model (`mps` for Mac GPUs, `cuda` for Nvidia GPUs, and `cpu` otherwise).

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from torch.backends.mps import is_available as is_mps_available
from torch.cuda import is_available as is_cuda_available

if is_mps_available():
    device = "mps"
elif is_cuda_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using {device}")

In [ ]:


embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5", device=device)
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

Finally, let's set up our RAG query engine.
If we want to perform simple question-answering, we can use the `as_query_engine` method.
If we want to perform chat with history, we can use the `as_chat_engine` method.
We can see both below 👇.


In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
llm = OpenAI(model="gpt-4o-mini")
Settings.llm = llm # set gpt-4o-mini as the default llm

query_engine = index.as_query_engine(llm=llm)
chat_engine = index.as_chat_engine(llm=llm)

Let's see how the query enine works.
We start by passing it a question, then it uses the retriever to find the most relevant documents.
Finally, it uses the LLM to answer the question.

In [ ]:
response = query_engine.query("How many new Llama models models are mentioned in the paper?")

Below we see the response text.
But there's also some additional information that we can access, including the source nodes.
These are the nodes that the retriever used to answer the question.
We can see that the retriever found several nodes that are relevant to the question, then the LLM used at least one of them to answer the question.

In [ ]:
print(response.response)

Taking a look at the first source node, we can see that it is a node that contains part of the document that is relevant to the question.
It also contains the `score`, which is the similarity between the question and the node.


In [ ]:
print(response.source_nodes[0])

Now let's see if our chat enigine comes up with the same answer.

In [ ]:
chat_response = chat_engine.chat("How many new Llama models models are mentioned in the paper?")
print(chat_response.response)
print(chat_response.source_nodes[0])

# Data ingestion

Data often comes in many different formats.
It may come in the form of a PDF, a web page, a code file, etc.
We may need some specific processing pipelines to extract the text from these documents, split them correctly, and vectorize them.

Luckily, LlamaIndex (and other libraries) provide lots of built-in and add-on tools to help you ingest almost any data type.
Instead of loading a PDF, let's load a web page instead.
We will use one of the classes provided by [`llama-index-readers-web`](https://llamahub.ai/l/readers/llama-index-readers-web?from=readers) to load data from a web page.

In this section, we will:
- Load a web page as Markdown
- Split it into chunks following the structured format of the Markdown
- Embed the chunks
- Store the chunks in a vector database
- Create a query engine from the vector database and use it to answer a question


In [ ]:
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb


In [ ]:
web_docs = SimpleWebPageReader(html_to_text=True).load_data(['https://en.wikipedia.org/wiki/Wikipedia'])

In [ ]:
wikipedia_collection = chromadb.EphemeralClient().create_collection("wikipedia", get_or_create=True)
wikipedia_vector_store = ChromaVectorStore(wikipedia_collection)

In [ ]:
wikipedia_pipeline = IngestionPipeline(
    transformations=[
        MarkdownNodeParser.from_defaults(),
        embed_model,
    ],
    vector_store=wikipedia_vector_store
)

In [ ]:
wikipedia_nodes = wikipedia_pipeline.run(documents=web_docs)

In [ ]:
wikipedia_index = VectorStoreIndex.from_vector_store(wikipedia_vector_store, embed_model=embed_model)

In [ ]:
llm = OpenAI(model="gpt-4o-mini")
query_engine = wikipedia_index.as_query_engine(llm=llm)
response = query_engine.query("How many languages there exactly? Quote the exact text as well.")
print(response.response)

In [ ]:
# Use fuzzywuzzy to find the closest match in the source text
from thefuzz import fuzz, process
# Get the top matching line of text from the source_text_quote
top_match, match_score = process.extractOne(response.response, response.source_nodes[0].text.splitlines(), scorer=fuzz.ratio)
assert top_match in response.source_nodes[0].text
print(f"Quote from source: '{top_match}'")

## OCR

There are times where the PDF is not in a format that can be easily read into text.
In these cases, we will need to use optical character recognition (OCR) to convert the images to text.
There are many libraries and cloud services that can do this, but for this example, we will use the `docling` library since our example document is a PDF.
We will then ingest the text into LlamaIndex and use it to answer a question.

![](https://ds4sd.github.io/docling/assets/docling_processing.png)

In [ ]:
from llama_index.readers.docling import DoclingReader
from llama_index.node_parser.docling import DoclingNodeParser
from llama_index.core.schema import Document
from llama_index.core import StorageContext

In [ ]:
reader = DoclingReader(export_type=DoclingReader.ExportType.JSON)

In [ ]:
docling_docs = reader.load_data(pdf_path)

In [ ]:
len(docling_docs)

In [ ]:
# We need to set the text template to "{content}" because the default is "{metadata}\n\n{content}",
# and LlamaIndex will try to embed the metadata as well. The metadata is not useful at serach time.
docling_docs[0].text_template
docling_docs[0].text_template = "{content}"

In [ ]:
docling_node_parser = DoclingNodeParser()
docling_nodes = docling_node_parser.get_nodes_from_documents(docling_docs)
len(docling_nodes)

In [ ]:
docling_nodes[0].metadata

In [ ]:
index = VectorStoreIndex(nodes=docling_nodes, embed_model=embed_model)

In [ ]:
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("How many new Llama models are mentioned in the paper?")
print(response.response)

In [ ]:
print(response.source_nodes[1])

# Data splitting

Many times, it's impractical to embed the entire document, and expensive to feed the entire document to the LLM.
Instead, we can split the document into chunks, embed the chunks, and use a retrieval method to find the most relevant chunks.
There are naïve methods that split texts into chunks of a specific length with some overlap;
there are methods that use the structure of the document to split it into sections (e.g. sections, figures, tables);
and there are more advanced methods that use semantic similarity to group the text into chunks.

Since the OCR'd text is just one very long Markdown string, we need to split it into chunks.
One nice way to do that is use the inherent structure of Markdown to split it into sections.
We do that here with LlamaIndex's `MarkdownNodeParser`.

In [ ]:
docling_md_docs = DoclingReader(export_type=DoclingReader.ExportType.MARKDOWN).load_data(pdf_path)

In [ ]:
from llama_index.core.node_parser import MarkdownNodeParser
from IPython.display import Markdown, display

In [ ]:
md_nodes = MarkdownNodeParser.from_defaults().get_nodes_from_documents(documents=docling_md_docs)

In [ ]:
len(md_nodes)

In [ ]:
display(Markdown(md_nodes[9].text))


In [ ]:
md_index = VectorStoreIndex(nodes=md_nodes, embed_model=embed_model)
md_query_engine = md_index.as_query_engine(llm=llm)
response = md_query_engine.query("How many llama3 models are there?")
print(response.response)
print(response.source_nodes[0].text)

---

# Retrieval

## Starting simple: bm25

BM25 is a simple retrieval method that uses the BM25 algorithm to score the relevance of each document to the query.
The BM25 algorithm is a probabilistic retrieval model that uses the term frequency and inverse document frequency of the query terms to score the relevance of each document.
You should be familiar with the basic idea of tf-idf from your NLP class - you can think of BM25 as a generalization of tf-idf that takes into account more factors.

In [ ]:
from llama_index.retrievers.bm25 import BM25Retriever

In [ ]:
bm25 = BM25Retriever.from_defaults(nodes=md_nodes)

In [ ]:
for node in bm25.retrieve("How many llama3 models are there?"):
    print(f"Score: {node.score:.4f}\nText:\n{node.node.text[:500]}...")

## Dense retrieval (vector search)

BM25 is a simple and fast method that depends on word matching.
But if we want to do more complex retrieval, we can use dense retrieval.
We represent both our query and documents as vectors and use a similarity metric to find the most relevant documents.
This is what's been going on under the hood in the previous examples using `VectorStoreIndex`.

Since most of the mechanics are taken care for us under the hood, let's examine what goes on under the hood.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
sample_documents = [
    "My favorite type of dog is a golden retriever.",
    "I like to eat pizza with my friends.",
    "I like to go to the gym in the morning.",
    "I like to play basketball with my friends.",
]

embeddings = np.array(embed_model.get_text_embedding_batch(sample_documents))

query = "What do I like to do with friends?"
query_embedding = np.array(embed_model.get_text_embedding(query))

In [ ]:
cosine_similarity(query_embedding.reshape(1, -1), embeddings).squeeze()

Once we have a `VectorStoreIndex`, we can use the `as_retriever` method to get a retriever object.
This uses dense retrieval under the hood, since it already has an embedding model as a part of the class.
Here, we use the `similarity_top_k` parameter to limit the number of results to 2 and show the cosine similarity score along with the beginning of the text.

In [ ]:
query = "How many llama3 models are there?"
top_results = md_index.as_retriever(similarity_top_k=2).retrieve(query)
for result in top_results:
    print(f"Score: {result.score:.4f}\nText:\n{result.node.text[:500]}...")

Let's compare bm25 with dense retrieval using %%timeit.

In [ ]:
%%timeit
bm25.retrieve(query)

In [ ]:
%%timeit
md_index.as_retriever(similarity_top_k=2).retrieve(query)

We can see that bm25 is much faster!

## Hybrid search: query rewriting and reciprocal ranking

Sometimes, you may want several methods of searching over your data, then combining the results.
This is known as hybrid search.

Haveing multiple retrievers may not mean having separate objects - we may just have multiple queries.
In this example, we'll use an LLM to rewrite our query into multiple queries, then use a dense retriever to find the most relevant documents.
Finally, we'll use reciprocal ranking to re-rank the results.


In [ ]:
from llama_index.core.retrievers import QueryFusionRetriever

In [ ]:
dense_retriever = md_index.as_retriever(similarity_top_k=5)
hybrid_retriever = QueryFusionRetriever(
    [dense_retriever],
    num_queries=3,
    use_async=False,
    mode='reciprocal_rerank',
    verbose=True
)

In [ ]:
results = hybrid_retriever.retrieve("How many llama3 models are there?")
for result in results:
    print(f"Score: {result.score:.4f}\nText:\n{result.node.text[:500]}...")

# Reranking

Often, retrieval only gives us a good first pass at finding relevant documents.
We can use re-ranking to improve the results.
There are several re-ranking methods, but in this case we'll use a cross-encoder to re-rank the results.
The simplest way to rerank is to use a `node_postprocessor` as we construct our query engine or chat engine.
Here, we'll use the `SentenceTransformerRerank` node postprocessor to re-rank the top 20 results from the dense retriever down to 5.

In [ ]:
from llama_index.core.postprocessor import SentenceTransformerRerank
md_query_engine = md_index.as_query_engine(llm=llm, node_postprocessors=[
    SentenceTransformerRerank(top_n=5)
], retriever_top_k=20)

In [ ]:
response = md_query_engine.query("How many llama3 models are there?")
print(response.response)

This is also a good opportunity to introduce workflows.
Workflows are a lower-level method for organizing tasks that can be chained together.
We'll see how to mimic the `md_query_engine` we made above using a workflow.

In [ ]:
from llama_index.core.schema import NodeWithScore
from llama_index.core.llms import ChatMessage
from llama_index.core.workflow import (
    Workflow,
    Context,
    step,
    StartEvent,
    StopEvent,
    Event
)
from typing import List


Let's walk throught the code below step by step.
First, we define the events that will be passed between the steps.
This includes a retrieval result, a re-ranking result, and a prompt for the LLM.
The start and stop are already taken care of for us by `StartEvent` and `StopEvent`.

Next, we define our `RAGWithReRank` workflow.
We initialize it with an index and LLM, which we will use for retrieval and answering and the cross-encoder for re-ranking.
We also define exactly what to do for each step of the workflow.
It knows what to do because of the `@step` decorator and the types annotations of each method.

Finally, we can run the workflow with a query.
The workflow will automatically handle passing the events between the steps, so we don't have to worry about that.
We can observe that the results are just what we expect as we saw from the previous examples.
One advantage of this method is that it's much more flexible - you can include arbitrary code, loops, conditionals, etc.

For more detailed documentation on workflows, see [here](https://docs.llamaindex.ai/en/stable/understanding/workflows/).

In [ ]:
# Define the events that will be passed between the steps
class RetrievalResult(Event):
    results: List[NodeWithScore]

class ReRankResult(Event):
    results: List[NodeWithScore]

class RagPrompt(Event):
    prompt: str

# Define the workflow
class RAGWithReRank(Workflow):
    # Initialize the workflow with an index and LLM
    def __init__(self, index: VectorStoreIndex = md_index, llm: OpenAI = OpenAI(model="gpt-4o-mini")):
        super().__init__(timeout=10, verbose=False)
        self.index = index
        self.llm = llm
        self.reranker = SentenceTransformerRerank(top_n=5)

    # The start method is called when the workflow is run.
    # This is the first step of the workflow that takes in a query and returns a retrieval result.
    # Since we want to use a course retriever, we use a top k of 20.
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> RetrievalResult:
        query = ev.query
        await ctx.set('query', query)
        results = self.index.as_retriever(similarity_top_k=20).retrieve(query)
        return RetrievalResult(results=results)

    # This step takes the retrieval results and re-ranks them.
    # Notice that the similarity top k is set to 5, so we only keep the top 5 results.
    # This is smaller than the retrieval step since this is about refining a smaller number of results.
    @step
    async def rerank(self, ctx: Context, ev: RetrievalResult) -> ReRankResult:
        results = ev.results
        query = await ctx.get('query')
        reranked = self.reranker.postprocess_nodes(nodes=results, query_str=query)
        return ReRankResult(results=reranked)

    # This step creates a prompt for the LLM.
    # It does this by joining the re-ranked results into a single string and formatting it with the query.
    @step
    async def create_prompt(self, ctx: Context, ev: ReRankResult) -> RagPrompt:
        reranked_results = ev.results
        await ctx.set('reranked_results', reranked_results)
        query = await ctx.get('query')
        reranked_str = '\n\n'.join(i.text for i in reranked_results)
        prompt = f"""\
Here is some relevant context:
--------------------------------
{reranked_str}
--------------------------------
Based on the above information and not prior knowledge, please answer the question.
Question:
{query}
"""
        return RagPrompt(prompt=prompt)

    # This step takes the prompt and uses the LLM to answer the question.
    # It also takes the re-ranked results and attaches them to the response in case we want to see the nodes.
    @step
    async def answer(self, ctx: Context, ev: RagPrompt) -> StopEvent:
        prompt = ev.prompt
        ranked_results = await ctx.get('reranked_results')
        messages = [
            ChatMessage.from_str(prompt)
        ]
        answer = await self.llm.achat(messages)
        result = {
            "response": answer,
            "source_nodes": ranked_results
        }
        return StopEvent(result=result)


In [ ]:
rag = RAGWithReRank()

In [ ]:
response = await rag.run(query="How many llama3 models are there?")

In [ ]:
response.keys()

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows
from IPython.core.display import HTML, display
draw_all_possible_flows(RAGWithReRank, filename="flow.html")

# Exercise: Chat with PDF (30 minutes)

Your goal in this exercise is to create a Gradio interface for a question-answering system.
Your application should:
- Use the query engine created above to answer questions about the uploaded PDF
- Display the question and answer in the UI. If using QueryEngine, use a question and answer format. If using ChatEngine, use a chat format.

If you need a challenge:
- Look at the LlamaIndex docs and add something new to the pipeline, such as [`HyDEQueryTransform`](https://docs.llamaindex.ai/en/stable/examples/query_transformations/HyDEQueryTransformDemo/)
- Use the `gr.File` component to allow the user to upload ANY pdf and ask question about it.


In [ ]:
# Your code here

# Agents

LLMs on their own have limited capabilities to perform tasks.
One way to augment that is to give them tools they can use.
These tools allow them to perform actions in the real world, such as searching the web or updating a database.
In this section, we'll build up an understanding for how agents work under the hood and how to use them.

## Function calling

LLMs are fine tuned to call functions with parameters based on the prompt.
This means you can provide information about one ore more functions you want to call and have the LLM decide which one to call.
This is powerful tool when working with LLMs because it can provide a lot of external data and skills not available to the LLM itself.

Let's go through a simple function calling example.

In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
import numpy as np
from rich import print
import json

In the cell below, we define a function that rolls a number of dice and returns the results.
LlamaIndex provides a `FunctionTool` class that makes it easy to convert a function into a tool.

In [ ]:
def roll_dice(num_dice:int) -> str:
    """
    Rolls a number of dice and returns the results.

    Args:
        num_dice: The number of dice to roll

    Returns:
        str: The results of the dice roll (comma separated if multiple dice are rolled)
    """
    return ', '.join(str(np.random.randint(1, 6)) for _ in range(num_dice))

roll_dice_tool = FunctionTool.from_defaults(fn=roll_dice, description="Useful when you need to roll dice.")

Let's ask our LLM to roll 5 dice and see what happens.

In [ ]:
llm = OpenAI(model="gpt-4o-mini")
messages = [
    ChatMessage.from_str("Roll 5 dice")
]
response = llm.chat_with_tools(tools=[roll_dice_tool], messages=messages)

It doesn't actually call the function yet - it just says which function to call and which arguments to pass to it.

In [ ]:
print(response)

Let's parse the response to get the function call and arguments.

In [ ]:
function_call = response.raw.choices[0].message.tool_calls[0].function
print(function_call)

Finally, let's call the function with the arguments.

In [ ]:
tool_output = roll_dice_tool.call(**json.loads(function_call.arguments))
print(tool_output)

Great! Now we have a basic idea of how function calling works under the hood.
Agents use function calling to call tools, but generally use them in loops combined with reasoning steps to accomplish complex tasks.
Next, we'll see how to use LlamaIndex's implementation of agents.

## Intro to Llama index agents: Financial Data Agent

We've seen how to use function calling with an LLM, but it was kind of difficult to go from the LLM response to the function call.
We didn't even close the loop and pass the tool call result back to the LLM.
LlamaIndex's `FunctionCallingAgent` and `ReActAgent` classes make this easier.
Let's go through an example of using the `FunctionCallingAgent` to perform a more complex tasks.

In [ ]:
from llama_index.core.agent import ReActAgent, FunctionCallingAgent
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import FunctionTool
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec
from llama_index.llms.openai import OpenAI
from yfinance import download as yf_download
from datetime import datetime

It's really useful to be able to define your own tools.
Luckliy, LlamaIndex makes this easy with the `FunctionTool` class.
Here, we'll define a tool that gets stock data from Yahoo Finance and returns it as a markdown table.

In [ ]:
# def get_todays_date() -> str:
#   """
#   Gets today's date
#   """

# today_tool = FunctionTool.from_defaults(fn=get_todays_date)

In [ ]:
def get_stock_data(ticker:str, start_date:str, end_date:str) -> str:
    """
    Gets stock data using yfinance. All dates should be in YYYY-MM-DD format.

    Args:
        ticker: The ticker symbol of the stock to get data for
        start_date: The start date of the data to get
        end_date: The end date of the data to get

    Returns:
        str: A markdown table of the stock data
    """
    df = yf_download(ticker, start=start_date, end=end_date)
    return df.to_markdown()

get_stock_data_tool = FunctionTool.from_defaults(fn=get_stock_data, description="Useful when you want to pull trended data about a stock.")

Now, we can pass this tool to an agent to have it answer questions about stocks.

In [ ]:
agent = FunctionCallingAgent.from_tools(tools=[get_stock_data_tool], llm=OpenAI(model="gpt-4o-mini"), verbose=True)
response = agent.chat(f"What was TSLA's high and low over the last 7 days?")

In [ ]:
print(response.response)

In [ ]:
agent = FunctionCallingAgent.from_tools(tools=[get_stock_data_tool], llm=OpenAI(model="gpt-4o-mini"), verbose=True)
response = agent.chat(f"What was TSLA's high and low over the last 7 days? Today's date is {datetime.today}")
print(response.response)

In [ ]:
agent = ReActAgent.from_tools(tools=[get_stock_data_tool], llm=OpenAI(model="gpt-4o-mini"), verbose=True)
response = agent.chat(f"What was TSLA's high and low over the last 7 days? Today's date is {datetime.today}")
print(response.response)

### Check for understanding

Is there anything strange about the response to this question above?
What do you think we could do to fix it?

## Using pre-packaged tools

LlamaIndex has a number of tools that are pre-defined and can be installed.
In this case, we will use the `YahooFinanceToolSpec` to get additional information about a stock beyond just the trended data.
We will also add our `get_stock_data_tool` to the list of tools, since getting trended data is not a part of the `YahooFinanceToolSpec`.

In [ ]:
yfinance_tools = YahooFinanceToolSpec().to_tool_list()
yfinance_tools.append(get_stock_data_tool)

In [ ]:
agent = FunctionCallingAgent.from_tools(tools=yfinance_tools, llm=OpenAI(model="gpt-4o-mini"), verbose=True)
response = agent.chat(f"What was TSLA's high and low over the last 7 days? Today's date is {datetime.now().strftime('%Y-%m-%d')}.")

In [ ]:
response = agent.chat("What does TSLA make?")

## Demo 2: Agentic RAG

In the previous examples of RAG, we had a single retriever that retrieved documents from a single index.
But what if we have multiple indices we want to use?
We can use an agent to decide which index to use and then retrieve from that index.
One great way to do this is to turn the indices we made previously into tools.
We'll make an agent that can answer questions about the llama3 paper OR the Wikipedia page we ingested earlier.

In [ ]:
from llama_index.core.tools import RetrieverTool

Let's create a tool for each of our indices.

In [ ]:
wikipedia_tool = RetrieverTool.from_defaults(
    retriever=wikipedia_index.as_retriever(),
    name="wikipedia_retriever",
    description="Useful when you need information about the Wikipedia organization."
)
llama_tool = RetrieverTool.from_defaults(
    retriever=md_index.as_retriever(),
    name="llama_retriever",
    description="Useful when you need information about the llama3 paper."
)

retriever_tools = [wikipedia_tool, llama_tool]

Now, we can pass these tools to an agent to have it answer questions about the Wikipedia page or the llama3 paper.
This time, instead of using the `FunctionCallingAgent`, we'll use the `OpenAIAgent`.

In [ ]:
rag_agent = OpenAIAgent.from_tools(tools=retriever_tools, llm=OpenAI(model="gpt-4o-mini"), verbose=True)

In [ ]:
rag_agent.query("When was Wikipedia founded?")

In [ ]:
rag_agent.query("How did llama3 8b perform on MMLU?")

## Demo 3: Code interpreter

In [ ]:
from llama_index.tools.code_interpreter import CodeInterpreterToolSpec

In [ ]:
code_tools = CodeInterpreterToolSpec().to_tool_list()
system_prompt = """\
You are a helpful assistant that can answer questions by executing python code. \
Always make sure to print the results to StdOut.
"""
code_agent = OpenAIAgent.from_tools(tools=code_tools, llm=OpenAI(model="gpt-4o-mini"), verbose=True, system_prompt=system_prompt)
code_agent.query("What is the square root of 9936.3981?")

### Check for understanding

What are some possible downsides and security concerns of using a code interpreter tool?

# Exercise: Build your own agent

The goal of this exercise is to build your own agent.


## Planning (15 minutes)

Your goal over the next 15 minutes is to answer the following questions:
- What is the goal of the agent? What task (or tasks) will it accomplish?
- What tools will it need to use to accomplish its goal?
- How will it use the tools to accomplish its goal?
- What is the modality of the agent? (for example, conversational, workflow, code writing, etc.)

If you need some inspiration, here are some ideas places you can go to learn about external tools:
- [Llam Index tools on LlamaHub](https://llamahub.ai/?tab=tools)
- [This repo with free APIs](https://github.com/public-api-lists/public-api-lists) (look for ones w/o auth since we don't have a ton of time for setup)

#### YOUR PLAN HERE

## Eric's Idea

Use


Agent: Stock Trading Assistant


1. Get the top 10 NASDAQ most active (pre-market) tickers and also 5 minutes after the market opens.
    1a. Premarket
    1b. After market opens today
2. Get past 1 years of daily data from yahoo finance.
    2a. Average for 40 days Compute if the trend is possitive or negative.
3. Get the % Change from the day before market close. 
4. Get the number of active shares issue.
5. Get the number of volume issue.
6. Get the number of price low and high from pre-market
7. Get the number of price low and high from yesterday
8. Get the number of price low and high from the last 5 minutes
9. Compute all the standard deviations.
10. Get the ticker with highest possitive %Change, the highest volume and number of shares, the highest standard deviation and has a positive trend for the last 1 year.
11. Send the result via whatsapp text.


You may use some of this tools if necessary:
from llama_index.core.agent import ReActAgent, FunctionCallingAgent
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import FunctionTool
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec
from llama_index.llms.openai import OpenAI
from yfinance import download as yf_download
from datetime import datetime


Agent: Stock Trading Assistant
The agent will analyze stock market data in real-time and send insights via WhatsApp.

Steps and Tasks
Step 1: Get Top 10 NASDAQ Most Active Tickers
1a: Premarket
Retrieve the top 10 most active NASDAQ tickers before the market opens.

1b: After Market Opens
Retrieve the top 10 most active NASDAQ tickers 5 minutes after the market opens.

Step 2: Fetch 1 Year of Historical Data
Daily Data
Retrieve the past year’s daily stock price data for the tickers obtained in Step 1 using Yahoo Finance.

40-Day Moving Average Trend
Compute the 40-day moving average for each ticker and determine if the trend is positive (upward) or negative (downward).

Step 3: Calculate % Change
Day-Over-Day % Change
Calculate the percentage change in the stock price compared to the previous day’s closing price.
Step 4: Retrieve Number of Active Shares
Get the total number of shares issued for each ticker.
Step 5: Retrieve Trading Volume
Retrieve the trading volume for each ticker.
Step 6: Get Premarket Price Highs and Lows
Identify the highest and lowest prices for each ticker during the premarket session.
Step 7: Get Yesterday’s Price Highs and Lows
Retrieve the highest and lowest prices for each ticker during the previous trading day.
Step 8: Get Last 5 Minutes’ Price Highs and Lows
Identify the highest and lowest prices for each ticker in the last 5 minutes of trading activity.
Step 9: Compute Standard Deviations (Real-Time)
Calculate the standard deviations of price values for each ticker based on:
Premarket highs and lows (Step 6).
Yesterday’s highs and lows (Step 7).
Last 5 minutes’ highs and lows (Step 8).
This step provides a measure of real-time price volatility.

Step 10: Identify the Best Ticker
Determine the stock ticker that meets all the following criteria:
Highest Positive % Change (from Step 3).
Highest Volume (from Step 5).
Highest Number of Shares Issued (from Step 4).
Highest Standard Deviation (from Step 9).
Positive 40-Day Trend (from Step 2).
Step 11: Send the Result via WhatsApp
Send a message summarizing the best ticker and relevant metrics (e.g., % change, volume, shares, trend, and standard deviation) using a WhatsApp API.
Tools & Libraries
LlamaIndex

Use ReActAgent or FunctionCallingAgent for step automation and task execution.
Leverage the YahooFinanceToolSpec for real-time stock data retrieval.
yfinance

Fetch premarket, historical, and real-time price data.
datetime

Manage time-based filtering for data (e.g., last 5 minutes).
WhatsApp API (e.g., Twilio)

Send notifications with the final results.

In [ ]:
brew install graphviz

In [ ]:
from graphviz import Digraph

# Create a directed graph to represent the workflow of the Stock Trading Assistant agent
workflow = Digraph(format='png', graph_attr={'rankdir': 'LR', 'fontsize': '12', 'fontname': 'Arial'})
workflow.attr('node', shape='box', fontsize='10', fontname='Arial')

# Step 1: Get NASDAQ Most Active Tickers
workflow.node("Step 1", "Step 1: Get NASDAQ Most Active Tickers\n1a. Premarket\n1b. 5 Minutes After Market Opens")

# Step 2: Fetch Historical Data
workflow.node("Step 2", "Step 2: Fetch 1-Year Historical Data\n- 40-Day Moving Average Trend")

# Step 3: Calculate % Change
workflow.node("Step 3", "Step 3: Calculate Day-Over-Day % Change")

# Step 4: Retrieve Active Shares
workflow.node("Step 4", "Step 4: Retrieve Number of Active Shares")

# Step 5: Retrieve Volume
workflow.node("Step 5", "Step 5: Retrieve Trading Volume")

# Step 6: Premarket Highs and Lows
workflow.node("Step 6", "Step 6: Get Premarket Price Highs and Lows")

# Step 7: Yesterday's Highs and Lows
workflow.node("Step 7", "Step 7: Get Yesterday's Price Highs and Lows")

# Step 8: Last 5 Minutes Highs and Lows
workflow.node("Step 8", "Step 8: Get Last 5 Minutes' Price Highs and Lows")

# Step 9: Compute Standard Deviations
workflow.node("Step 9", "Step 9: Compute Standard Deviations\n- Premarket, Yesterday, Last 5 Minutes")

# Step 10: Identify Best Ticker
workflow.node("Step 10", "Step 10: Identify Best Ticker\n- Highest % Change\n- Highest Volume\n- Highest Shares\n- Highest Std. Dev.\n- Positive Trend")

# Step 11: Send Result via WhatsApp
workflow.node("Step 11", "Step 11: Send Result via WhatsApp")

# Connect the nodes to represent the workflow
workflow.edges([
    ("Step 1", "Step 2"),
    ("Step 2", "Step 3"),
    ("Step 3", "Step 4"),
    ("Step 4", "Step 5"),
    ("Step 5", "Step 6"),
    ("Step 6", "Step 7"),
    ("Step 7", "Step 8"),
    ("Step 8", "Step 9"),
    ("Step 9", "Step 10"),
    ("Step 10", "Step 11")
])



In [ ]:
# Save the diagram as a PNG
output_path = "./stock_trading_assistant_workflow"
workflow.render(output_path, format='png', cleanup=True)


## Backend building (30 minutes)

Use LlamaIndex agents to build out your agent.
Your goal over the next 30 minutes is to:
- Create (or import) any tools you need to accomplish your goal.
- Create your agent using one of the agent classes provided by LlamaIndex.
- Create 2-3 test cases and check your agent against them to see if it produces the desired output.

In [ ]:
# Your code here

In [ ]:
pip show llama-index

In [ ]:
!pip install yfinance

In [ ]:
from llama_index.core.agent import ReActAgent, FunctionCallingAgent
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import FunctionTool

import yfinance as yf

from llama_index.tools.yahoo_finance import YahooFinanceToolSpec


from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import requests

import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')



# Function to send a message via WhatsApp using an API (e.g., Twilio)
def send_whatsapp_message(message, recipient_number):
    # Twilio or equivalent API details
    API_URL = "https://api.example.com/send-message"
    API_TOKEN = "your_api_token"
    
    payload = {
        "to": recipient_number,
        "message": message,
    }
    headers = {"Authorization": f"Bearer {API_TOKEN}"}
    response = requests.post(API_URL, json=payload, headers=headers)
    return response.status_code

# Step 1: Get NASDAQ Most Active Tickers
def get_most_active_tickers(period="1d"):
    nasdaq_data = yf.download("^IXIC", period=period, interval="1m")
    return nasdaq_data.index[:10]  # Simulated: Replace with actual most active tickers logic

# Step 2: Fetch 1-Year Historical Data
def fetch_historical_data(tickers):
    data = {}
    for ticker in tickers:
        stock_data = yf.download(ticker, period="1y")
        data[ticker] = stock_data
    return data

def compute_40_day_trend(data):
    trends = {}
    for ticker, df in data.items():
        df['40D_MA'] = df['Close'].rolling(window=40).mean()
        trends[ticker] = "Positive" if df['40D_MA'].iloc[-1] > df['40D_MA'].iloc[0] else "Negative"
    return trends

# Step 3: Calculate % Change
def calculate_percent_change(data):
    percent_changes = {}
    for ticker, df in data.items():
        yesterday_close = df['Close'].iloc[-2]
        today_open = df['Open'].iloc[-1]
        percent_changes[ticker] = ((today_open - yesterday_close) / yesterday_close) * 100
    return percent_changes

# Step 4 & 5: Get Active Shares and Volume
def get_active_shares_and_volume(tickers):
    results = {}
    for ticker in tickers:
        info = yf.Ticker(ticker).info
        results[ticker] = {
            "shares_outstanding": info.get("sharesOutstanding", 0),
            "volume": info.get("volume", 0),
        }
    return results

# Step 6, 7 & 8: Price Highs and Lows
def get_price_highs_and_lows(tickers, timeframe):
    results = {}
    for ticker in tickers:
        stock_data = yf.download(ticker, period=timeframe)
        results[ticker] = {
            "high": stock_data['High'].max(),
            "low": stock_data['Low'].min(),
        }
    return results

# Step 9: Compute Standard Deviations
def compute_standard_deviations(price_data):
    std_devs = {}
    for ticker, data in price_data.items():
        std_devs[ticker] = np.std([data["high"], data["low"]])
    return std_devs

# Step 10: Identify Best Ticker
def identify_best_ticker(percent_changes, volumes, active_shares, std_devs, trends):
    best_ticker = None
    highest_score = -1

    for ticker in percent_changes.keys():
        if trends[ticker] == "Positive":
            score = (
                percent_changes[ticker] +
                volumes[ticker]["volume"] +
                active_shares[ticker]["shares_outstanding"] +
                std_devs[ticker]
            )
            if score > highest_score:
                highest_score = score
                best_ticker = ticker
    return best_ticker

# Step 11: Send Results via WhatsApp
def notify_result(best_ticker, percent_changes, volumes, active_shares, trends, recipient_number):
    message = (
        f"Best Ticker: {best_ticker}\n"
        f"% Change: {percent_changes[best_ticker]:.2f}%\n"
        f"Volume: {volumes[best_ticker]['volume']}\n"
        f"Shares Outstanding: {active_shares[best_ticker]['shares_outstanding']}\n"
        f"Trend: {trends[best_ticker]}"
    )
    send_whatsapp_message(message, recipient_number)

# Main Execution Flow
def stock_trading_assistant(recipient_number):
    # Step 1
    tickers = get_most_active_tickers()
    
    # Step 2
    historical_data = fetch_historical_data(tickers)
    trends = compute_40_day_trend(historical_data)
    
    # Step 3
    percent_changes = calculate_percent_change(historical_data)
    
    # Step 4 & 5
    volumes_and_shares = get_active_shares_and_volume(tickers)
    
    # Step 6, 7, 8
    premarket_data = get_price_highs_and_lows(tickers, "1d")
    yesterday_data = get_price_highs_and_lows(tickers, "5d")
    last_5_min_data = get_price_highs_and_lows(tickers, "1m")
    
    # Step 9
    std_devs = compute_standard_deviations({
        ticker: {
            "high": premarket_data[ticker]["high"],
            "low": last_5_min_data[ticker]["low"]
        }
        for ticker in tickers
    })
    
    # Step 10
    best_ticker = identify_best_ticker(
        percent_changes,
        volumes_and_shares,
        volumes_and_shares,
        std_devs,
        trends
    )
    
    # Step 11
    notify_result(best_ticker, percent_changes, volumes_and_shares, volumes_and_shares, trends, recipient_number)

# Run the agent
recipient = "+1234567890"  # Replace with the actual recipient number
stock_trading_assistant(recipient)


In [5]:
from datetime import datetime, timedelta

# Example of date manipulation
start_date = datetime.now() - timedelta(days=365)
end_date = datetime.now()

print("Start Date:", start_date.strftime('%Y-%m-%d'))
print("End Date:", end_date.strftime('%Y-%m-%d'))

# Ensure yfinance handles these dates
ticker = "AAPL"
stock_data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
print(stock_data.tail())


Start Date: 2023-12-09

End Date: 2024-12-08

[*********************100%***********************]  1 of 1 completed


Price        Adj Close       Close        High         Low        Open  \
Ticker            AAPL        AAPL        AAPL        AAPL        AAPL   
Date                                                                     
2024-12-02  239.589996  239.589996  240.789993  237.160004  237.270004   
2024-12-03  242.649994  242.649994  242.759995  238.899994  239.809998   
2024-12-04  243.009995  243.009995  244.110001  241.250000  242.869995   
2024-12-05  243.039993  243.039993  244.539993  242.130005  243.990005   
2024-12-06  242.839996  242.839996  244.630005  242.080002  242.910004   

Price         Volume  
Ticker          AAPL  
Date                  
2024-12-02  48137100  
2024-12-03  38861000  
2024-12-04  44383900  
2024-12-05  40033900  
2024-12-06  36852100

In [13]:
tickers = get_most_active_tickers()

[*********************100%***********************]  1 of 1 completed


In [ ]:
print(tickers)

Price                         Adj Close         Close          High  \
Ticker                            ^IXIC         ^IXIC         ^IXIC   
Datetime                                                              
2024-12-06 14:30:00+00:00  19737.169922  19737.169922  19749.605469   
2024-12-06 14:31:00+00:00  19741.789062  19741.789062  19757.300781   
2024-12-06 14:32:00+00:00  19750.146484  19750.146484  19753.339844   
2024-12-06 14:33:00+00:00  19766.419922  19766.419922  19766.419922   
2024-12-06 14:34:00+00:00  19767.306641  19767.306641  19770.794922   
...                                 ...           ...           ...   
2024-12-06 20:55:00+00:00  19858.599609  19858.599609  19858.832031   
2024-12-06 20:56:00+00:00  19858.898438  19858.898438  19858.898438   
2024-12-06 20:57:00+00:00  19858.304688  19858.304688  19859.369141   
2024-12-06 20:58:00+00:00  19856.341797  19856.341797  19860.593750   
2024-12-06 20:59:00+00:00  19854.685547  19854.685547  19861.406250   

Price                               Low          Open    Volume  
Ticker                            ^IXIC         ^IXIC     ^IXIC  
Datetime                                                         
2024-12-06 14:30:00+00:00  19735.029297  19749.333984         0  
2024-12-06 14:31:00+00:00  19741.789062  19743.939453  51354161  
2024-12-06 14:32:00+00:00  19741.773438  19741.773438  37316648  
2024-12-06 14:33:00+00:00  19753.582031  19756.626953  35532966  
2024-12-06 14:34:00+00:00  19766.128906  19769.099609  33629523  
...                                 ...           ...       ...  
2024-12-06 20:55:00+00:00  19856.210938  19856.210938  19884000  
2024-12-06 20:56:00+00:00  19853.410156  19858.603516  32001000  
2024-12-06 20:57:00+00:00  19856.140625  19857.101562  54960000  
2024-12-06 20:58:00+00:00  19855.343750  19858.902344  17769000  
2024-12-06 20:59:00+00:00  19854.685547  19854.845703  35179000  

[388 rows x 6 columns]

In [6]:


# Main Execution Flow
def stock_trading_assistant(recipient_number):
    # Step 1
    tickers = get_most_active_tickers()

    # Step 2
    historical_data = fetch_historical_data(tickers)
    trends = compute_40_day_trend(historical_data)

    # Step 3
    percent_changes = calculate_percent_change(historical_data)

    # Step 4 & 5
    volumes_and_shares = get_active_shares_and_volume(tickers)

    # Step 6, 7, 8
    premarket_data = get_price_highs_and_lows(tickers, "1d")
    yesterday_data = get_price_highs_and_lows(tickers, "5d")
    last_5_min_data = get_price_highs_and_lows(tickers, "1m")

    # Step 9
    std_devs = compute_standard_deviations({
        ticker: {
            "high": premarket_data[ticker]["high"],
            "low": last_5_min_data[ticker]["low"]
        }
        for ticker in tickers
    })

    # Step 10
    best_ticker = identify_best_ticker(
        percent_changes,
        volumes_and_shares,
        volumes_and_shares,
        std_devs,
        trends
    )

    # Step 11
    notify_result(best_ticker, percent_changes, volumes_and_shares, volumes_and_shares, trends, recipient_number)

# Run the agent
recipient = "+1234567890"  # Replace with the actual recipient number
stock_trading_assistant(recipient)


[*********************100%***********************]  1 of 1 completed


Error fetching data for 2024-12-06 14:30:00+00:00: value must be an integer, received <class 'str'> for year

Error fetching data for 2024-12-06 14:31:00+00:00: value must be an integer, received <class 'str'> for year

Error fetching data for 2024-12-06 14:32:00+00:00: value must be an integer, received <class 'str'> for year

Error fetching data for 2024-12-06 14:33:00+00:00: value must be an integer, received <class 'str'> for year

...

Error fetching data for 2024-12-06 14:36:00+00:00: value must be an integer, received <class 'str'> for year

Error fetching data for 2024-12-06 14:37:00+00:00: value must be an integer, received <class 'str'> for year

Error fetching data for 2024-12-06 14:38:00+00:00: value must be an integer, received <class 'str'> for year

Error fetching data for 2024-12-06 14:39:00+00:00: value must be an integer, received <class 'str'> for year

AttributeError: 'Timestamp' object has no attribute 'upper'

## Frontend building (15 minutes)

Your goal over the next 15 minutes is to:
- Use `gradio` to build an interface for your agent.
- Test your agent with 2-3 different inputs to ensure it's working as expected.
- If you want to use your agent more like an api, check out the [`gradio-client`](https://www.gradio.app/docs/python-client/introduction) library and run your test cases programmatically.

In [ ]:
# Your code here

In [9]:
import gradio as gr
import yfinance as yf
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

# Core functions for the Stock Trading Assistant
def stock_trading_assistant(recipient_number):
    # Step 1: Get the NASDAQ most active tickers (for simplicity, using hardcoded tickers here)
    tickers = ["AAPL", "MSFT", "TSLA", "GOOGL", "AMZN"]  # Replace with dynamic retrieval

    # Step 2: Fetch historical data
    historical_data = fetch_historical_data(tickers)
    trends = compute_40_day_trend(historical_data)

    # Step 3: Calculate percentage change
    percent_changes = calculate_percent_change(historical_data)

    # Step 4 & 5: Get active shares and volume
    volumes_and_shares = get_active_shares_and_volume(tickers)

    # Step 6, 7, 8: Price highs and lows
    premarket_data = get_price_highs_and_lows(tickers, "1d")
    yesterday_data = get_price_highs_and_lows(tickers, "5d")
    last_5_min_data = get_price_highs_and_lows(tickers, "1m")

    # Step 9: Compute standard deviations
    std_devs = compute_standard_deviations({
        ticker: {
            "high": premarket_data[ticker]["high"],
            "low": last_5_min_data[ticker]["low"]
        }
        for ticker in tickers
    })

    # Step 10: Identify the best ticker
    best_ticker = identify_best_ticker(
        percent_changes,
        volumes_and_shares,
        volumes_and_shares,
        std_devs,
        trends
    )

    # Result message
    message = (
        f"Best Ticker: {best_ticker}\n"
        f"% Change: {percent_changes[best_ticker]:.2f}%\n"
        f"Volume: {volumes_and_shares[best_ticker]['volume']}\n"
        f"Shares Outstanding: {volumes_and_shares[best_ticker]['shares_outstanding']}\n"
        f"Trend: {trends[best_ticker]}"
    )

    return message

# Gradio interface
def assistant_interface(recipient_number):
    # Run the stock trading assistant
    result = stock_trading_assistant(recipient_number)
    return result

# Define Gradio inputs and outputs
with gr.Blocks() as interface:
    gr.Markdown("## Stock Trading Assistant")

    with gr.Row():
        recipient_number_input = gr.Textbox(
            label="Enter Recipient Phone Number",
            placeholder="+1234567890"
        )

    with gr.Row():
        run_button = gr.Button("Run Stock Trading Assistant")

    with gr.Row():
        result_output = gr.Textbox(label="Result", interactive=False)

    run_button.click(assistant_interface, inputs=[recipient_number_input], outputs=[result_output])

# Launch Gradio interface
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a24bc09b594794b116.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install gradio-client

In [ ]:
from gradio_client import Client

client = Client("http://localhost:7860/")  # Use the URL from the Gradio app
response = client.predict("+1234567890", api_name="default")
print(response)